In [1]:
import pandas as pd


In [ ]:
API_KEY = ''
URL = 'https://datamall2.mytransport.sg/ltaodataservice/BusStops'


In [3]:
import requests
import json

# Make API call with API key in headers
headers = {
    'AccountKey': API_KEY,
    'accept': 'application/json'
}

# Make the request
response = requests.get(URL, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Parse JSON response
    data = response.json()
    
    # Get the list of bus stops (usually under 'value' key for LTA APIs)
    if 'value' in data:
        bus_stops = data['value']
        print(f"Total bus stops returned: {len(bus_stops)}")
        print(f"\nFirst 10 bus stops:\n")
        print(json.dumps(bus_stops[:10], indent=2))
    else:
        # If structure is different, print the first 10 items
        print("Response structure:")
        print(json.dumps(data, indent=2)[:2000])  # First 2000 chars
        if isinstance(data, list):
            print(f"\nFirst 10 items:")
            print(json.dumps(data[:10], indent=2))
else:
    print(f"Error: API returned status code {response.status_code}")
    print(f"Response: {response.text}")


Total bus stops returned: 500

First 10 bus stops:

[
  {
    "BusStopCode": "01012",
    "RoadName": "Victoria St",
    "Description": "Hotel Grand Pacific",
    "Latitude": 1.29684825487647,
    "Longitude": 103.85253591654006
  },
  {
    "BusStopCode": "01013",
    "RoadName": "Victoria St",
    "Description": "St. Joseph's Ch",
    "Latitude": 1.29770970610083,
    "Longitude": 103.8532247463225
  },
  {
    "BusStopCode": "01019",
    "RoadName": "Victoria St",
    "Description": "Bras Basah Cplx",
    "Latitude": 1.29698951191332,
    "Longitude": 103.85302201172507
  },
  {
    "BusStopCode": "01029",
    "RoadName": "Nth Bridge Rd",
    "Description": "Opp Natl Lib",
    "Latitude": 1.2966729849642,
    "Longitude": 103.85441422464267
  },
  {
    "BusStopCode": "01039",
    "RoadName": "Nth Bridge Rd",
    "Description": "Bugis Cube",
    "Latitude": 1.29820784139683,
    "Longitude": 103.85549139837407
  },
  {
    "BusStopCode": "01059",
    "RoadName": "Victoria St",
    "

In [4]:
# Fetch all bus stops (handle pagination) and create CSV with SVY21 coordinates
import requests
import pandas as pd

# Install pyproj if not available
try:
    from pyproj import Transformer
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyproj"])
    from pyproj import Transformer

# Setup API headers
headers = {
    'AccountKey': API_KEY,
    'accept': 'application/json'
}

# Create transformer for WGS84 to SVY21 conversion
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3414", always_xy=True)

# Fetch all bus stops (LTA API returns 500 at a time, use $skip for pagination)
all_bus_stops = []
skip = 0
batch_size = 500

print("Fetching all bus stops from API...")

while True:
    # Add $skip parameter for pagination
    url_with_skip = f"{URL}?$skip={skip}"
    response = requests.get(url_with_skip, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: API returned status code {response.status_code}")
        print(f"Response: {response.text}")
        break
    
    data = response.json()
    
    if 'value' in data:
        batch = data['value']
        if len(batch) == 0:
            break  # No more data
        all_bus_stops.extend(batch)
        print(f"Fetched {len(batch)} bus stops (total so far: {len(all_bus_stops)})")
        
        if len(batch) < batch_size:
            break  # Last batch
        skip += batch_size
    else:
        print("Unexpected response structure")
        break

print(f"\nTotal bus stops fetched: {len(all_bus_stops)}")

# Convert to DataFrame
df_bus_stops = pd.DataFrame(all_bus_stops)

# Rename columns to match requirements
df_bus_stops = df_bus_stops.rename(columns={
    'BusStopCode': 'BusStopID',
    'Longitude': 'Longitude',
    'Latitude': 'Latitude'
})

# Reorder columns: BusStopID, RoadName, Description, Longitude, Latitude
df_bus_stops = df_bus_stops[['BusStopID', 'RoadName', 'Description', 'Longitude', 'Latitude']]

# Convert coordinates to SVY21
def convert_to_svy21(row):
    """Convert WGS84 lon/lat to SVY21 X/Y"""
    lon = row['Longitude']
    lat = row['Latitude']
    
    # Skip if coordinates are missing
    if pd.isna(lon) or pd.isna(lat):
        return pd.Series({'SVY21_X': None, 'SVY21_Y': None})
    
    # Transform coordinates
    x, y = transformer.transform(lon, lat)
    
    return pd.Series({'SVY21_X': x, 'SVY21_Y': y})

# Apply conversion
print("\nConverting coordinates to SVY21...")
svy21_coords = df_bus_stops.apply(convert_to_svy21, axis=1)
df_bus_stops['SVY21_X'] = svy21_coords['SVY21_X']
df_bus_stops['SVY21_Y'] = svy21_coords['SVY21_Y']

# Save to CSV
df_bus_stops.to_csv('bus_stops.csv', index=False)
print(f"\n✓ CSV file saved as 'bus_stops.csv'")
print(f"\nTotal bus stops: {len(df_bus_stops)}")
print(f"\nColumns: {', '.join(df_bus_stops.columns.tolist())}")
print(f"\nFirst 5 rows:")
print(df_bus_stops.head().to_string(index=False))


Fetching all bus stops from API...
Fetched 500 bus stops (total so far: 500)
Fetched 500 bus stops (total so far: 1000)
Fetched 500 bus stops (total so far: 1500)
Fetched 500 bus stops (total so far: 2000)
Fetched 500 bus stops (total so far: 2500)
Fetched 500 bus stops (total so far: 3000)
Fetched 500 bus stops (total so far: 3500)
Fetched 500 bus stops (total so far: 4000)
Fetched 500 bus stops (total so far: 4500)
Fetched 500 bus stops (total so far: 5000)
Fetched 179 bus stops (total so far: 5179)

Total bus stops fetched: 5179

Converting coordinates to SVY21...

✓ CSV file saved as 'bus_stops.csv'

Total bus stops: 5179

Columns: BusStopID, RoadName, Description, Longitude, Latitude, SVY21_X, SVY21_Y

First 5 rows:
BusStopID      RoadName         Description  Longitude  Latitude      SVY21_X      SVY21_Y
    01012   Victoria St Hotel Grand Pacific 103.852536  1.296848 30138.719949 31024.417898
    01013   Victoria St     St. Joseph's Ch 103.853225  1.297710 30215.379876 31119.673